<a href="https://colab.research.google.com/github/BIRDRED/datascience/blob/main/Projeto_Violencia_Clusterizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
from sklearn import linear_model as lm
from sklearn import preprocessing, model_selection
from sklearn import metrics
from scipy import stats
from scipy.stats import norm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import pickle
import warnings

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets


%matplotlib inline



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#PRÉ-PROCESSAMENTO

##CARREGAMENTO DE ARQUIVOS

In [3]:
df_municipio = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/Municipio.csv', sep=";")

In [4]:
df_homicidio = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-homicidios.csv', sep=";")

In [5]:
df_homicidio_jovens = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-de-homicidio-de-jovens-por-armas-de-fogo.csv', sep=";")

In [6]:
df_suicidio = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-suicidios-por-armas-de-fogo.csv', sep=";")

In [7]:
df_causaindeterminada = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-mortes-violentas-por-causa-indeterminada.csv', sep=";")

In [8]:
df_suicidio_homens = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-de-suicidio-de-homens.csv', sep=";")

In [9]:
df_suicidio_mulheres = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-de-suicidio-de-mulheres.csv', sep=";")

In [10]:
df_intervencao_legal = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-intervencao-legal.csv', sep=";")

In [11]:
df_homicidio_15a29_anos = pd.read_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/taxa-homicidio-15a29anos.csv', sep=";")

## JUNÇÃO DOS DATAFRAMES

In [12]:
df_violencia = pd.merge(df_homicidio,df_homicidio_jovens,on=['cod','período','nome'], how='left')

In [13]:
df_violencia = pd.merge(df_violencia,df_suicidio,on=['cod','período','nome'], how='left')

In [14]:
df_violencia = pd.merge(df_violencia,df_causaindeterminada,on=['cod','período','nome'], how='left')

In [15]:
df_violencia = pd.merge(df_violencia,df_suicidio_homens,on=['cod','período','nome'], how='left')

In [16]:
df_violencia = pd.merge(df_violencia,df_suicidio_mulheres,on=['cod','período','nome'], how='left')

In [17]:
df_violencia = pd.merge(df_violencia,df_intervencao_legal,on=['cod','período','nome'], how='left')

In [18]:
df_violencia = pd.merge(df_violencia,df_homicidio_15a29_anos,on=['cod','período','nome'], how='left')

In [19]:
df_violencia.to_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/Violencia.csv',sep=";",index=False)

## AJUSTES DA TABELA

In [20]:
df_violencia_total = pd.merge(df_municipio,df_violencia,on=['cod'], how='left')

In [21]:
df_violencia_total = df_violencia_total.set_index(['cod'])

In [22]:
df_violencia_total.head()

,municipio,microrregiao/id,microrregiao/nome,microrregiao/mesorregiao/id,microrregiao/mesorregiao/nome,microrregiao/mesorregiao/UF/id,microrregiao/mesorregiao/UF/sigla,microrregiao/mesorregiao/UF/nome,microrregiao/mesorregiao/UF/regiao/id,microrregiao/mesorregiao/UF/regiao/sigla,...,nome,período,homicidios,homicidio jovens armas de fogo,suicidio arma de fogo,violencia por causa indeterminada,suicidio homens,suicidio mulheres,intervencao_legal,homicidio_15a29
cod,,,,,,,,,,,,,,,,,,,,,
1100015,Alta Floresta D'Oeste,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,N,...,Alta Floresta D'Oeste,2007.0,7.84,0.00,0.0,3.92,7.53,0.00,0.0,13.88
1100015,Alta Floresta D'Oeste,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,N,...,Alta Floresta D'Oeste,2008.0,43.45,28.02,0.0,0.00,30.37,8.23,0.0,56.04
1100015,Alta Floresta D'Oeste,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,N,...,Alta Floresta D'Oeste,2009.0,15.91,0.00,0.0,3.97,7.65,8.28,0.0,0.00
1100015,Alta Floresta D'Oeste,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,N,...,Alta Floresta D'Oeste,2010.0,28.05,28.69,0.0,4.00,15.43,0.00,0.0,43.03
1100015,Alta Floresta D'Oeste,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,N,...,Alta Floresta D'Oeste,2011.0,12.13,14.58,0.0,0.00,0.00,8.39,0.0,29.17


In [23]:
df_violencia_total = df_violencia_total.drop(['microrregiao/id', 'microrregiao/nome',
       'microrregiao/mesorregiao/id', 'microrregiao/mesorregiao/nome',
       'microrregiao/mesorregiao/UF/id',
       'microrregiao/mesorregiao/UF/nome',
       'microrregiao/mesorregiao/UF/regiao/id',
       'microrregiao/mesorregiao/UF/regiao/nome', 'regiao-imediata/id',
       'regiao-imediata/nome', 'regiao-imediata/regiao-intermediaria/id',
       'regiao-imediata/regiao-intermediaria/nome',
       'regiao-imediata/regiao-intermediaria/UF/id',
       'regiao-imediata/regiao-intermediaria/UF/nome',
       'regiao-imediata/regiao-intermediaria/UF/regiao/id',
       'regiao-imediata/regiao-intermediaria/UF/regiao/nome', 'nome','regiao-imediata/regiao-intermediaria/UF/regiao/sigla','regiao-imediata/regiao-intermediaria/UF/sigla'],axis=1)

In [24]:
df_violencia_total.columns

Index(['municipio', 'microrregiao/mesorregiao/UF/sigla',
       'microrregiao/mesorregiao/UF/regiao/sigla', 'período', 'homicidios',
       'homicidio jovens armas de fogo', 'suicidio arma de fogo',
       'violencia por causa indeterminada', 'suicidio homens',
       'suicidio mulheres', 'intervencao_legal', 'homicidio_15a29'],
      dtype='object')

In [25]:
df_violencia_total.rename(columns = {'microrregiao/mesorregiao/UF/sigla':'UF', 'microrregiao/mesorregiao/UF/regiao/sigla':'Regiao'}, inplace = True)


In [26]:
df_violencia_total = df_violencia_total[df_violencia_total['homicidios'].notnull()]

In [27]:
df_violencia_total.to_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/Violencia_Periodos.csv',sep=";",index=False)

# CLUSTERIZAÇÃO

##FUNÇÕES

In [28]:
from matplotlib import cm
from sklearn.metrics import silhouette_samples

In [29]:
def elbow(values):
  wcss = []

  for i in range(1, 11):
      kmeans = KMeans(n_clusters = i)
      kmeans.fit(values)
      wcss.append(kmeans.inertia_)

  plt.plot(range(1, 11), wcss)
  plt.title('The elbow method')
  plt.xlabel('Number of clusters')
  plt.ylabel('WCSS') #within cluster sum of squares
  plt.show()

In [30]:
def silhouette(values, type=None):
  if type=='bar':
    silhouette_scores = []
    for i in range(2,10):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit_predict(values)
        silhouette_scores.append(silhouette_score(values, kmeans.fit_predict(values))) 
    plt.bar(range(2,10), silhouette_scores) 
    plt.xlabel('Número de clusters') 
    plt.ylabel('Score') 
    plt.show()
  else:
    for n_clusters in range(2,11):
      fig, axes = plt.subplots(1, 2, figsize=(20,5))
      clusterer = KMeans(n_clusters=n_clusters)
      cluster_labels = clusterer.fit_predict(values)
      silhouette_values = silhouette_samples(values, cluster_labels)
      y_lower = 10
      for i in range(n_clusters):
        ith_cluster_silhouette_values = silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        # METODO 'nipy_spectral' altera o colormap padrao
        color = cm.nipy_spectral(float(i) / n_clusters)

        axes[0].fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)
        
        # SE QUISER MOSTRAR O LABEL DO CLUSTER
        #axes[0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  
        axes[0].set_title("The silhouette plot for the various clusters.")
        axes[0].set_xlabel("The silhouette coefficient values")
        axes[0].set_ylabel("Cluster label")
        silhouette_avg = silhouette_score(values, cluster_labels)
        axes[0].axvline(x=silhouette_avg, color="red", linestyle="--")
        axes[0].set_yticks([]) 
        axes[0].set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        axes[1].scatter(values[:, 2], values[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                    c=colors, edgecolor='k')    

        ## SE QUISER IMPRIMIR O CENTRO
        centers = clusterer.cluster_centers_
        #axes[1].scatter(centers[:, 1], centers[:, 2], marker='o', c="white", alpha=1, s=200, edgecolor='k')
        
        #for i, c in enumerate(centers):
         #   axes[1].scatter(c[1], c[2], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')
        axes[1].set_title("The visualization of the clustered data.")
        axes[1].set_xlabel("Feature space for the 1st feature")
        axes[1].set_ylabel("Feature space for the 2nd feature")
        plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                      "with n_clusters = %d" % n_clusters),
                    fontsize=14, fontweight='bold')

    plt.show()

In [31]:
def cluster(dataset):
  scaler = StandardScaler()
  ds_Players = scaler.fit_transform(dataset)
  Players_scaled = pd.DataFrame(ds_Players, columns=dataset.columns)
  elbow(ds_Players)
  silhouette(ds_Players)
  return Players_scaled

In [32]:
def cluster_kmeans(dataset,dataset_original,x):
  kmeans = KMeans(n_clusters=x)
  ykmeans = kmeans.fit_predict(dataset)
  dataset['clusters'] = ykmeans
  dataset_original['clusters'] = ykmeans
  return dataset,dataset_original

## Gerar Cluster

In [33]:
df_violencia_total.columns

Index(['municipio', 'UF', 'Regiao', 'período', 'homicidios',
       'homicidio jovens armas de fogo', 'suicidio arma de fogo',
       'violencia por causa indeterminada', 'suicidio homens',
       'suicidio mulheres', 'intervencao_legal', 'homicidio_15a29'],
      dtype='object')

In [34]:
df_violencia_total = df_violencia_total.drop(['municipio', 'UF', 'Regiao','período'],axis=1)

In [35]:
#cluster(df_violencia_total)

In [36]:
cluster_kmeans(df_violencia_total,df_violencia,2)

(         homicidios  homicidio jovens armas de fogo  suicidio arma de fogo  \
 cod                                                                          
 1100015        7.84                            0.00                   0.00   
 1100015       43.45                           28.02                   0.00   
 1100015       15.91                            0.00                   0.00   
 1100015       28.05                           28.69                   0.00   
 1100015       12.13                           14.58                   0.00   
 ...             ...                             ...                    ...   
 5300108       20.81                           34.48                   0.37   
 5300108       17.82                           28.22                   0.94   
 5300108       17.82                           28.22                   0.94   
 5300108       15.92                           22.68                   0.79   
 5300108       15.92                           22.68

In [37]:
df_violencia[df_violencia['clusters'] == 0]

,cod,nome,período,homicidios,homicidio jovens armas de fogo,suicidio arma de fogo,violencia por causa indeterminada,suicidio homens,suicidio mulheres,intervencao_legal,homicidio_15a29,clusters
21,2512200,Prata,2007,26.16,99.90,0.00,0.00,0.00,52.4,0.0,99.90,0
22,2608206,Joaquim Nabuco,2007,49.41,62.20,6.17,6.17,36.98,0.0,0.0,82.94,0
23,2512101,Pombal,2007,3.05,11.37,0.00,0.00,6.37,0.0,0.0,11.37,0
24,2512002,Pocinhos,2007,17.83,22.05,0.00,0.00,0.00,0.0,0.0,22.05,0
25,2511905,Pitimbu,2007,6.04,0.00,0.00,6.03,0.00,12.2,0.0,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
120033,4315800,Roca Sales,2019,8.78,43.57,8.77,0.00,3.00,17.3,0.0,43.57,0
120035,4315909,Rodeio Bonito,2019,51.13,171.67,0.00,17.00,0.00,0.0,0.0,171.67,0
120036,4315909,Rodeio Bonito,2019,51.13,171.67,0.00,17.00,0.00,0.0,0.0,171.67,0
120037,4315958,Rolador,2019,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0


In [38]:
dictCluster = {0:'Muito Violento' ,1: 'Pouco Violento'}

In [39]:
df_violencia['TipoCluster'] = df_violencia['clusters'].map(dictCluster)

In [40]:
df_violencia['TipoCluster'].value_counts()

Pouco Violento    99991
Muito Violento    20060
Name: TipoCluster, dtype: int64

##Gerar Arquivo - Cluster

In [41]:
df_violencia.to_csv('/content/drive/MyDrive/Pós-Graduação/Aprendizados de Dados em Tempo Real/Projetos/Violencia_Clusters.csv',sep=";",index=False)